#OpenPrompt Installation
OpenPrompt is a modular and flexible platform to develop a prompt-learning pipeline

In [ ]:
!pip install openprompt

#Data Pre-Processing

Importing Data from separate csv files.
The datasets are available on https://github.com/samin9796/arg2keypoint.

Note: There are different versions of trainning datasets, for whole training set, for few-shot and for subset experiments 

In [ ]:
import pandas as pd
train_df = pd.read_csv("train.csv")
dev_df = pd.read_csv("dev.csv")
test_df = pd.read_csv("test.csv")

For **fewshot** and **subset** experiments the following line helps to filter out some extra columns:

In [ ]:
# train_df = train_df.filter(['Unnamed: 0', 'topic', 'argument', 'key_point','stance', 'label'], axis=1)

In [ ]:

train_df.columns=['idx', 'topic', 'argument', 'key_point', 'stance', 'label']
dev_df.columns=['idx', 'topic', 'argument', 'key_point', 'stance', 'label']
test_df.columns=['idx', 'topic', 'argument', 'key_point', 'stance', 'label']


To merge train, dev, and test dataframes into one dataset:

In [ ]:
from openprompt.data_utils import InputExample
dataset = {}

dataset['train'] = []
dataset['dev'] = []
dataset['test'] = []
for index, data in train_df.iterrows():
  input_example = InputExample(text_a = data['argument'], text_b = data['key_point'], label=int(data['label']), guid=data['idx'])
  dataset['train'].append(input_example)
for index, data in dev_df.iterrows():
  input_example = InputExample(text_a = data['argument'], text_b = data['key_point'], label=int(data['label']), guid=data['idx'])
  dataset['dev'].append(input_example)
for index, data in test_df.iterrows():
  input_example = InputExample(text_a = data['argument'], text_b = data['key_point'], label=int(data['label']), guid=data['idx'])
  dataset['test'].append(input_example)


#Obtain a Pre-trained Language Model

In [ ]:
from openprompt.plms import load_plm
plm, tokenizer, model_config, WrapperClass = load_plm("t5", "t5-base")

#Define a Template
There are five different templates in the first approach. In each experiment we use one of them. You need to uncomment each of them that you want to use.

###Manual Templates


Template 1: 'The argument: [X1] and the keypoint [X2] are [Z].'


In [ ]:
from openprompt.prompts import ManualTemplate
template_text = 'The argument: {"placeholder": "text_a"}, and the keypoint: {"placeholder": "text_b"} are {"mask"}'
mytemplate = ManualTemplate(tokenizer=tokenizer, text=template_text)

Template 2: 'The argument: [X1] is [Z] with the keypoint: [X2]'


In [ ]:
# template_text = 'The argument: {"placeholder": "text_a"} is {"mask"} with the keypoint: {"placeholder": "text_b"}'
# mytemplate = ManualTemplate(tokenizer=tokenizer, text=template_text)

Template 3: 'Does the argument: [X1] comprise the fact that [X2]? [Z]'

In [ ]:
# template_text = 'Does the argument: {"placeholder": "text_a"}, comprise the fact that: {"placeholder": "text_b"}? {"mask"}'
# mytemplate = ManualTemplate(tokenizer=tokenizer, text=template_text)

Template 4: 'A keypoint is a summarization of the corresponding argument. In other words, an argument comprises a keypoint. Does the argument: [X1], comprise the keypoint [X2]? [Z]'


In [ ]:
# template_text = 'A keypoint is a summarization of the corresponding argument. In other words, an argument comprises a keypoint. Does the argument: {"placeholder": "text_a"}, comprise the the keypoint: {"placeholder": "text_b"}? {"mask"}'
# mytemplate = ManualTemplate(tokenizer=tokenizer, text=template_text)

###Mixed Template
In MixedTemplate, you can use {"soft"} to denote a tunable template.

Template 5: 'Argument: [X1] Keypoint: [X2] {"soft": "Does"} {"soft": "the", "soft_id": 1} argument matches {"soft_id": 1} keypoint? [Z]'

In [ ]:
# from openprompt.prompts import MixedTemplate

# mytemplate = MixedTemplate(model=plm, tokenizer=tokenizer, text='Argument: {"placeholder": "text_a"} Keypiont: {"placeholder": "text_b"} {"soft": "Does"} {"soft": "the", "soft_id": 1} argument matches {"soft_id": 1} keypoint? {"mask"}')

In [ ]:
wrapped_example = mytemplate.wrap_one_example(dataset['train'][0])
print(wrapped_example)

##Tokenization

In [ ]:
wrapped_t5tokenizer = WrapperClass(max_seq_length=128, decoder_max_length=3, tokenizer=tokenizer,truncate_method="head")

In [ ]:
tokenized_example = wrapped_t5tokenizer.tokenize_one_example(wrapped_example, teacher_forcing=False)
print(tokenized_example)
print(tokenizer.convert_ids_to_tokens(tokenized_example['input_ids']))
print(tokenizer.convert_ids_to_tokens(tokenized_example['decoder_input_ids']))

In [ ]:
model_inputs = {}
for split in ['train', 'dev', 'test']:
    model_inputs[split] = []
    for sample in dataset[split]:
        tokenized_example = wrapped_t5tokenizer.tokenize_one_example(mytemplate.wrap_one_example(sample), teacher_forcing=False)
        model_inputs[split].append(tokenized_example)



In [ ]:
from openprompt import PromptDataLoader

train_dataloader = PromptDataLoader(dataset=dataset["train"], template=mytemplate, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=256, decoder_max_length=3,
    batch_size=4,shuffle=True, teacher_forcing=False, predict_eos_token=False,
    truncate_method="head")

# Define a Verbalizer
 In classification, you need to define your verbalizer, which is a mapping from logits on the vocabulary to the final label probability.

For example the verbalizer contains multiple label words in each class. We have two different vocabulary as final labels:

For **template 1** and **template 2** use the following verbalizer:

In [ ]:
from openprompt.prompts import ManualVerbalizer
import torch

myverbalizer = ManualVerbalizer(tokenizer, num_classes=2,
                        label_words=[["matched"], ["not matched"]])



For **template 3** , **template 4**, and **template 5** uncomment and use the following verbalizer:

In [ ]:
# from openprompt.prompts import ManualVerbalizer
# import torch

# myverbalizer = ManualVerbalizer(tokenizer, num_classes=2,
#                         label_words=[["Yes"], ["No"]])

Creating a pseudo output from the plm, and see what the verbalizer do:

In [ ]:
print(myverbalizer.label_words_ids)
logits = torch.randn(2,len(tokenizer)) 
print(myverbalizer.process_logits(logits)) 

In [ ]:
from openprompt import PromptForClassification

use_cuda = True
prompt_model = PromptForClassification(plm=plm,template=mytemplate, verbalizer=myverbalizer, freeze_plm=False)
if use_cuda:
    prompt_model=  prompt_model.cuda()

##Training

In [ ]:
from transformers import  AdamW, get_linear_schedule_with_warmup
loss_func = torch.nn.CrossEntropyLoss()

no_decay = ['bias', 'LayerNorm.weight']

# it's always good practice to set no decay to biase and LayerNorm parameters
optimizer_grouped_parameters1 = [
    {'params': [p for n, p in prompt_model.plm.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in prompt_model.plm.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

# Using different optimizer for prompt parameters and model parameters
optimizer_grouped_parameters2 = [
    {'params': [p for n,p in prompt_model.template.named_parameters() if "raw_embedding" not in n]}
]

optimizer1 = AdamW(optimizer_grouped_parameters1, lr=1e-4)
optimizer2 = AdamW(optimizer_grouped_parameters2, lr=1e-3)

for epoch in range(3):
    tot_loss = 0
    for step, inputs in enumerate(train_dataloader):
        if use_cuda:
            inputs = inputs.cuda()
        logits = prompt_model(inputs)
        labels = inputs['label']
        loss = loss_func(logits, labels)
        loss.backward()
        tot_loss += loss.item()
        optimizer1.step()
        optimizer1.zero_grad()
        optimizer2.step()
        optimizer2.zero_grad()
        # print(tot_loss/(step+1))


#Evaluate

In [ ]:

validation_dataloader = PromptDataLoader(dataset=dataset["test"], template=mytemplate, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=256, decoder_max_length=3,
    batch_size=4,shuffle=False, teacher_forcing=False, predict_eos_token=False,
    truncate_method="head")

allpreds = []
alllabels = []
for step, inputs in enumerate(validation_dataloader):
    if use_cuda:
        inputs = inputs.cuda()
    logits = prompt_model(inputs)
    labels = inputs['label']
    alllabels.extend(labels.cpu().tolist())
    allpreds.extend(torch.argmax(logits, dim=-1).cpu().tolist())

acc = sum([int(i==j) for i,j in zip(allpreds, alllabels)])/len(allpreds)
print(acc)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(alllabels, allpreds, digits=3))